In [1]:
from autostore import Database, GeometryRow, CalculationRow, EnergyRow
from qcio import Results

db = Database()

res = Results.model_validate(
        {
            "input_data": {
                "structure": {
                    "symbols": ["O", "H", "H"],
                    "geometry": [
                        [0.0, 0.0, 0.0],
                        [1.8897261259082012, 0.0, 0.0],
                        [0.0, 1.8897261259082012, 0.0],
                    ],
                    "charge": 0,
                    "multiplicity": 1,
                },
                "model": {"method": "gfn2", "basis": None},
                "calctype": "energy",
            },
            "success": True,
            "data": {"energy": -5.062316802835694},
            "provenance": {"program": "crest", "program_version": "3.0.2"},
        }
    )

geom = GeometryRow.from_results(res)
calc = CalculationRow.from_results(res)

with db.session() as session:
    session.add(geom)
    session.add(calc)
    session.commit()

    session.refresh(geom)
    session.refresh(calc)

    energy = EnergyRow(
        value=res.data.energy,
        geometry=geom,
        calculation=calc,
    )

    session.add(energy)
    session.commit()